In [ ]:
from __init__ import *
from MultiResSolver_ct import *

from skimage.io import imread
import cv2

In [ ]:
box_basis = BoxBasis()

In [ ]:
img = imread('data/ID_0000_AGE_0060_CONTRAST_1_CT.tif') 
img = (img - np.min(img)) / (img.max() - img.min())
image_size = 255
dim = (image_size, image_size)

# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)    

    
device = 'cuda:1'
N_grid = 257
N_theta = 120
N_y = 200
thetas = torch.linspace(0, np.pi, N_theta).to(device).double()
thetas = thetas[:N_theta]
y_values = torch.linspace(-400, 400, N_y)[:, None, None]

c_tensor = F.pad(torch.from_numpy(img), (1, 1, 1, 1))[None, None, :, :].to(device)

x_ray = XrayMS(N_y, N_theta, box_basis, thetas, y_values, device)
x_ray.update_grid(1, N_grid)
y = x_ray.H(c_tensor)

In [ ]:
mrs = MultiResSolver('ct', 'htv', lmbda =0.1, h_init=8, N_scales=5, range_r=256,
                 device=device, verbose=True, toi=1e-6, N_rays=N_y, N_theta=N_theta, box_basis=box_basis, thetas=thetas, y_values=y_values)

In [ ]:
mrs.solve_ct(y)

In [ ]:
rep = 'saved_results/'
torch.save(mrs.sols[0], rep+'mrs_0_htv_0.1.pt')
torch.save(mrs.sols[1], rep+'mrs_1_htv_0.1.pt')
torch.save(mrs.sols[2], rep+'mrs_2_htv_0.1.pt')
torch.save(mrs.sols[3], rep+'mrs_3_htv_0.1.pt')
torch.save(mrs.sols[4], rep+'mrs_4_htv_0.1.pt')

In [ ]:
torch.save(mrs.costs, rep+'mrs_htv_cost_0.1.pt')
torch.save(mrs.losses, rep+'mrs_htv_loss_0.1.pt')
torch.save(mrs.reg_values, rep+'mrs_htv_reg_0.1.pt')
torch.save(mrs.mses, rep+'mrs_htv_mse_0.1.pt')
torch.save(mrs.iters, rep+'mrs_htv_iters_0.1.pt')